In [2]:
from flow_dataframe import flow_from_dataframe

import numpy as np
import pandas as pd
import os
import sys
from itertools import cycle

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

ModuleNotFoundError: No module named 'numpy'

In [2]:
# with tf.device('/cpu:0'):
#     model = load_model('results/mae_11/bone_age_weights.best.hdf5')

model = load_model('results/mae_11/bone_age_weights.best.hdf5')

In [3]:
class_str_col = 'boneage'
gender_str_col = 'male'

IMG_SIZE = (299, 299)
BATCH_SIZE = 32


In [4]:
base_bone_dir = os.path.join('rsna-bone-age')
boneage_df = pd.read_csv(os.path.join(base_bone_dir, 'boneage-test-dataset.csv'))
boneage_df['path'] = boneage_df['fileName'].map(lambda x: os.path.join(base_bone_dir,
                                                         'boneage-test-dataset', 
                                                         'boneage-test-dataset', 
                                                         '{}'.format(x)))

In [5]:
boneage_df['exists'] = boneage_df['path'].map(os.path.exists)
print(boneage_df['exists'].sum(), 'images found of', boneage_df.shape[0], 'total')

1262 images found of 1262 total


In [6]:
boneage_df[gender_str_col] = boneage_df[gender_str_col].map(
    lambda x: np.array([1]) if x else np.array([0])
    ) # map boolean values to 1 and 0

In [7]:
img_gen = ImageDataGenerator()

In [8]:
# used a fixed dataset for evaluating the algorithm
gen_boneage = flow_from_dataframe(img_gen, boneage_df, path_col='path',
                                        target_size=IMG_SIZE,
                                        color_mode='rgb',
                                        batch_size=BATCH_SIZE)

flow_from_dataframe() -->
## Ignore next message from keras, values are replaced anyways
Found 1262 images belonging to 1 classes.
Reinserting dataframe: 1262 images
flow_from_dataframe() <--


In [12]:
def combined_generators(image_generator, gender_data, batch_size):
    gender_generator = cycle(batch(gender_data, batch_size))
    while True:
        nextImage = next(image_generator)
        nextGender = next(gender_generator)
        assert len(nextImage[0]) == len(nextGender)
        yield [nextImage[0], nextGender], nextImage[1]

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [13]:
gen_wrapper = combined_generators(gen_boneage, boneage_df[gender_str_col], BATCH_SIZE)

In [14]:
history = model.predict_generator(
    gen_wrapper,
    steps=len(gen_boneage),
    workers=8,
    use_multiprocessing=True,
    verbose=1)

40/40 [==============================] - 61s 2s/step


In [16]:
history

[[133.60597]
 [130.72531]
 [133.54732]
 ...
 [135.35689]
 [132.83064]
 [131.94247]]
